# Algoritma 16.1 (Preconditioned CG for Reduced System)

In [2]:
import numpy as np

def preconditioned_cg(Z, G, Wzz, c2, x0, tol=1e-6, max_iter=1000):
    """
    Implements Algorithm 16.1: Preconditioned CG for Reduced Systems.
    
    Parameters:
    Z : ndarray
        Matrix Z.
    G : ndarray
        Matrix G.
    Wzz : ndarray
        Preconditioner (assumed to be invertible).
    c2 : ndarray
        Vector c2.
    x0 : ndarray
        Initial point x.
    tol : float, optional
        Convergence tolerance.
    max_iter : int, optional
        Maximum number of iterations.
    
    Returns:
    x : ndarray
        Solution vector.
    """
    x = x0.copy()
    r_z = Z.T @ G @ Z @ x + c2  # Compute initial residual
    g_z = np.linalg.solve(Wzz, r_z)  # Apply preconditioner
    d_z = -g_z  # Initial direction
    
    for _ in range(max_iter):
        alpha = (r_z.T @ g_z) / (d_z.T @ Z.T @ G @ Z @ d_z)  # Step size
        x = x + alpha * d_z  # Update x
        r_z_new = r_z + alpha * (Z.T @ G @ Z @ d_z)  # Compute new residual
        
        if np.linalg.norm(r_z_new) < tol:
            break  # Convergence check
        
        g_z_new = np.linalg.solve(Wzz, r_z_new)  # Apply preconditioner
        beta = (r_z_new.T @ g_z_new) / (r_z.T @ g_z)  # Compute beta
        d_z = -g_z_new + beta * d_z  # Update search direction
        
        r_z = r_z_new
        g_z = g_z_new
    
    return x


# Algoritma 16.2 (Projected CG Method)

In [4]:
def projected_cg(G, P, c, A, b, x0, tol=1e-6, max_iter=1000):
    """
    Projected Conjugate Gradient (CG) Method.
    
    Parameters:
        G : numpy.ndarray
            Symmetric positive definite matrix.
        P : function
            Projection operator that ensures feasibility.
        c : numpy.ndarray
            Constant vector.
        A : numpy.ndarray
            Matrix defining constraints Ax = b.
        b : numpy.ndarray
            Right-hand side of constraints.
        x0 : numpy.ndarray
            Initial feasible point satisfying Ax = b.
        tol : float, optional
            Convergence tolerance (default: 1e-6).
        max_iter : int, optional
            Maximum number of iterations (default: 1000).

    Returns:
        x : numpy.ndarray
            Solution vector.
    """
    x = x0
    r = G @ x + c
    g = P(r)
    d = -g
    
    for _ in range(max_iter):
        Gd = G @ d
        alpha = (r.T @ g) / (d.T @ Gd)
        x = x + alpha * d
        r_new = r + alpha * Gd
        g_new = P(r_new)
        beta = (r_new.T @ g_new) / (r.T @ g)
        d = -g_new + beta * d
        r = r_new
        g = g_new

        if np.linalg.norm(r) < tol:
            break
            
    return x


# Algoritma 16.3 (Active-Set Method for Convex QP)

In [6]:
def solve_qp_active_set(Q, c, A, b, E, I, tol=1e-6, max_iter=100):
    """
    Solves a convex quadratic programming (QP) problem using the Active-Set Method.
    
    Minimize: (1/2) x^T Q x + c^T x
    Subject to: Ax = b, i in E, Ax >= b, i in I
    
    Parameters:
        Q (numpy.ndarray): Positive semi-definite matrix of the quadratic term.
        c (numpy.ndarray): Linear term vector.
        A (numpy.ndarray): Constraint matrix.
        b (numpy.ndarray): Constraint vector.
        E (list): Index of equality constraint.
        I (list): Index of inequality constraint.
        tol (float): Tolerance for stopping criteria.
        max_iter (int): Maximum number of iterations.
    
    Returns:
        x (numpy.ndarray): Optimal solution.
    """
    m, n = A.shape  # Number of constraints and variables
    
    # Define a feasible point x
    
    x = x0.copy()
    W = W0.copy()
    
    for k in range(max_iter):
        # Solve Q p_k = -c with active constraints
        Z = np.eye(n)  # Assume identity for simplicity, should be computed properly
        p = np.linalg.solve(Q, -c)  # Newton step direction
        
        if np.linalg.norm(p) < tol:
            # Compute Lagrange multipliers
            lambdas = np.linalg.lstsq(A[W, :], -c, rcond=None)[0]
            
            if np.all(lambdas >= 0):
                return x  # Optimal solution found
            
            # Remove the most negative Lagrange multiplier
            j = np.argmin(lambdas)
            W.remove(W[j])
        else:
            # Compute step size
            alpha = 1.0  # Should compute based on constraints
            
            x_new = x + alpha * p
            
            # Check for blocking constraints
            blocking_constraints = [i for i in range(m) if A[i, :] @ x_new > b[i] + tol]
            if blocking_constraints:
                W.append(blocking_constraints[0])
            
            x = x_new
    
    return x  # Return last computed x if max_iter is reached


# Algoritma 16.4 (Predictor-Corrector Algorithm for QP)

In [8]:
def predictor_corrector_qp(x0, y0, lambd0, solve_kkt_system, max_iters=100, tol=1e-6):
    """
    Predictor-Corrector Algorithm for Quadratic Programming.
    
    Parameters:
    x0, y0, lambd0: Initial primal and dual variables (assumed to be feasible and positive).
    solve_kkt_system: Function that solves the KKT system given inputs.
    max_iters: Maximum number of iterations.
    tol: Convergence tolerance.
    """
    x, y, lambd = x0, y0, lambd0
    m = len(y)
    
    for k in range(max_iters):
        # Solve for affine direction (σ = 0)
        dx_aff, dy_aff, dlambda_aff = solve_kkt_system(x, y, lambd, sigma=0)
        
        # Compute step size for affine direction
        alpha_aff = max_alpha(y, lambd, dy_aff, dlambda_aff)
        
        # Compute μ and μ_aff
        mu = np.dot(y, lambd) / m
        mu_aff = (np.dot(y + alpha_aff * dy_aff, lambd + alpha_aff * dlambda_aff)) / m
        
        # Compute centering parameter σ
        sigma = (mu_aff / mu) ** 3
        
        # Solve for corrector direction
        dx, dy, dlambda = solve_kkt_system(x, y, lambd, sigma)
        
        # Choose step size τ
        tau_x = choose_tau(y, lambd, dy, dlambda)
        
        # Update variables
        x += tau_x * dx
        y += tau_x * dy
        lambd += tau_x * dlambda
        
        # Check convergence
        if np.linalg.norm(dx) < tol and np.linalg.norm(dy) < tol and np.linalg.norm(dlambda) < tol:
            break
    
    return x, y, lambd

def max_alpha(y, lambd, dy, dlambda):
    """Compute maximum step size α_aff"""
    alpha_y = np.min([-y[i] / dy[i] for i in range(len(y)) if dy[i] < 0] + [1])
    alpha_lambda = np.min([-lambd[i] / dlambda[i] for i in range(len(lambd)) if dlambda[i] < 0] + [1])
    return min(alpha_y, alpha_lambda, 1)

def choose_tau(y, lambd, dy, dlambda):
    """Choose step size τ according to α_pri and α_dual"""
    alpha_pri = max_alpha(y, lambd, dy, dlambda)
    alpha_dual = max_alpha(y, lambd, dy, dlambda)
    return min(alpha_pri, alpha_dual)

# Note: The function `solve_kkt_system(x, y, lambd, sigma)` needs to be defined separately based on the problem structure.


# Algoritma 16.5 (Gradient Projection Method for QP)

In [10]:
def gradient_projection_qp(Q, c, A, b, x0, tol=1e-6, max_iter=100):
    """
    Gradient Projection Method for Quadratic Programming (QP)
    Minimize: 0.5 * x^T Q x + c^T x
    Subject to: Ax <= b
    
    Parameters:
    Q : numpy.ndarray
        Quadratic term matrix (must be positive semi-definite)
    c : numpy.ndarray
        Linear term vector
    A : numpy.ndarray
        Constraint matrix
    b : numpy.ndarray
        Constraint vector
    x0 : numpy.ndarray
        Initial feasible point
    tol : float
        Convergence tolerance
    max_iter : int
        Maximum number of iterations
    
    Returns:
    x : numpy.ndarray
        Optimal solution
    """
    def kkt_conditions_satisfied(x, grad):
        return np.linalg.norm(grad) < tol
    
    def projection_onto_feasible_set(x):
        from scipy.optimize import linprog
        res = linprog(c=np.zeros_like(x), A_ub=A, b_ub=b, bounds=[(None, None)]*len(x), method='highs')
        return res.x if res.success else x
    
    x = x0.copy()
    for k in range(max_iter):
        grad = Q @ x + c
        if kkt_conditions_satisfied(x, grad):
            return x  # Stop if KKT conditions are met
        
        # Compute the Cauchy point
        pk = -grad
        alpha = 1.0  # Simple step size (can be improved with line search)
        xc = x + alpha * pk
        
        # Projection step to find a feasible approximate solution
        x_plus = projection_onto_feasible_set(xc)
        
        # Ensure descent
        if 0.5 * x_plus.T @ Q @ x_plus + c.T @ x_plus <= 0.5 * xc.T @ Q @ xc + c.T @ xc:
            x = x_plus
        else:
            break  # If no improvement, stop
    
    return x  # Return last feasible solution

# Example Usage
Q = np.array([[2, 0], [0, 2]])  # Example positive semi-definite matrix
c = np.array([-2, -5])  # Linear term
A = np.array([[1, 1], [-1, 2], [2, 1]])  # Constraint matrix
b = np.array([2, 2, 3])  # Constraint vector
x0 = np.array([0, 0])  # Initial feasible point

optimal_x = gradient_projection_qp(Q, c, A, b, x0)
print("Optimal Solution:", optimal_x)


Optimal Solution: [0. 1.]
